In [19]:
# See covid_virtual_env.sh for environment setup
import boto3, pandas as pd

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_rows', 500)

# Load Data

Set the client for boto3 (amazons toolkit) to read s3

In [3]:
client = boto3.client('s3')

In [7]:
# New York Times Data
obj = client.get_object(Bucket='covid19-lake', Key='enigma-nytimes-data-in-usa/csv/us_county/us_county.csv')
data = pd.read_csv(obj['Body'])

# Data Exploration

## Fundamental exploration
What values are showing up?

In [8]:
# Quick look at the records
data.head()

,date,county,state,fips,cases,deaths
0,2020-01-21,Snohomish,Washington,53061.0,1,0
1,2020-01-22,Snohomish,Washington,53061.0,1,0
2,2020-01-23,Snohomish,Washington,53061.0,1,0
3,2020-01-24,Cook,Illinois,17031.0,1,0
4,2020-01-24,Snohomish,Washington,53061.0,1,0


In [9]:
data.dtypes

date       object
county     object
state      object
fips      float64
cases       int64
deaths      int64
dtype: object

In [10]:
# Fix the date data type
data['date'] = data['date'].astype('datetime64')

In [11]:
data.dtypes

date      datetime64[ns]
county            object
state             object
fips             float64
cases              int64
deaths             int64
dtype: object

In [12]:
data.head()

,date,county,state,fips,cases,deaths
0,2020-01-21,Snohomish,Washington,53061.0,1,0
1,2020-01-22,Snohomish,Washington,53061.0,1,0
2,2020-01-23,Snohomish,Washington,53061.0,1,0
3,2020-01-24,Cook,Illinois,17031.0,1,0
4,2020-01-24,Snohomish,Washington,53061.0,1,0


In [13]:
# Unique iso2 codes
print(data.state.unique())

['Washington' 'Illinois' 'California' 'Arizona' 'Massachusetts'
 'Wisconsin' 'Texas' 'Nebraska' 'Utah' 'Oregon' 'Florida' 'New York'
 'Rhode Island' 'Georgia' 'New Hampshire' 'North Carolina' 'New Jersey'
 'Colorado' 'Maryland' 'Nevada' 'Tennessee' 'Hawaii' 'Indiana' 'Kentucky'
 'Minnesota' 'Oklahoma' 'Pennsylvania' 'South Carolina'
 'District of Columbia' 'Kansas' 'Missouri' 'Vermont' 'Virginia'
 'Connecticut' 'Iowa' 'Louisiana' 'Ohio' 'Michigan' 'South Dakota'
 'Arkansas' 'Delaware' 'Mississippi' 'New Mexico' 'North Dakota' 'Wyoming'
 'Alaska' 'Maine' 'Alabama' 'Idaho' 'Montana' 'Puerto Rico'
 'Virgin Islands' 'Guam' 'West Virginia' 'Northern Mariana Islands']


In [14]:
# Looks to have an extra day compared to JH
print(data.date.unique())

['2020-01-21T00:00:00.000000000' '2020-01-22T00:00:00.000000000'
 '2020-01-23T00:00:00.000000000' '2020-01-24T00:00:00.000000000'
 '2020-01-25T00:00:00.000000000' '2020-01-26T00:00:00.000000000'
 '2020-01-27T00:00:00.000000000' '2020-01-28T00:00:00.000000000'
 '2020-01-29T00:00:00.000000000' '2020-01-30T00:00:00.000000000'
 '2020-01-31T00:00:00.000000000' '2020-02-01T00:00:00.000000000'
 '2020-02-02T00:00:00.000000000' '2020-02-03T00:00:00.000000000'
 '2020-02-04T00:00:00.000000000' '2020-02-05T00:00:00.000000000'
 '2020-02-06T00:00:00.000000000' '2020-02-07T00:00:00.000000000'
 '2020-02-08T00:00:00.000000000' '2020-02-09T00:00:00.000000000'
 '2020-02-10T00:00:00.000000000' '2020-02-11T00:00:00.000000000'
 '2020-02-12T00:00:00.000000000' '2020-02-13T00:00:00.000000000'
 '2020-02-14T00:00:00.000000000' '2020-02-15T00:00:00.000000000'
 '2020-02-16T00:00:00.000000000' '2020-02-17T00:00:00.000000000'
 '2020-02-18T00:00:00.000000000' '2020-02-19T00:00:00.000000000'
 '2020-02-20T00:00:00.000

## Some basic group-by's
How many records, cases, and deaths are showing up for those values?

In [15]:
# iso2 to iso3 mappings
data.groupby(['state'])['state'].count()

state
Alabama                     3201
Alaska                       571
Arizona                      850
Arkansas                    3381
California                  3281
Colorado                    2913
Connecticut                  494
Delaware                     200
District of Columbia          65
Florida                     3526
Georgia                     7609
Guam                          57
Hawaii                       261
Idaho                       1502
Illinois                    4196
Indiana                     4447
Iowa                        3817
Kansas                      3092
Kentucky                    4730
Louisiana                   3258
Maine                        819
Maryland                    1318
Massachusetts                919
Michigan                    3767
Minnesota                   3519
Mississippi                 4022
Missouri                    4254
Montana                     1340
Nebraska                    2250
Nevada                       590
New 

In [18]:
# quick look at NY
## NYC counties are also not broken out, cant see queens, richmond, kings, new york seperately
max_date = data.date.max()
data[(data.state == 'New York') & (data.date == max_date)].groupby('county')[['cases','deaths']].sum()

,cases,deaths
county,,
Albany,1432,109
Allegany,36,0
Broome,373,28
Cattaraugus,60,1
Cayuga,58,1
Chautauqua,41,2
Chemung,131,5
Chenango,109,2
Clinton,76,4


In [22]:
# US Death Counts
data.groupby('date')[['cases','deaths']].sum()

,cases,deaths
date,,
2020-01-21,1,0
2020-01-22,1,0
2020-01-23,1,0
2020-01-24,2,0
2020-01-25,3,0
2020-01-26,5,0
2020-01-27,5,0
2020-01-28,5,0
2020-01-29,5,0


In [23]:
# New York Deaths Over Time
data[data['state']=='New York'].groupby('date')[['cases','deaths']].sum()

,cases,deaths
date,,
2020-03-01,1,0
2020-03-02,1,0
2020-03-03,2,0
2020-03-04,11,0
2020-03-05,22,0
2020-03-06,44,0
2020-03-07,89,0
2020-03-08,106,0
2020-03-09,142,0
